In [1]:
import os
import torch
import random
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchsummary import summary
from torchvision.models import resnet18
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Define constants
image_size = 224
batch_size = 8
num_epochs = 32
learning_rate = 0.001
num_classes = 4
dropout = 0.5

# Load the images in the training directory
train_dir = "./../../Project 3/data/Data_cleaning_step1_2/Train"
test_dir = "./../../Project 3/data/Data_cleaning_step1_2/Test"

# Define the transformations for train data before entering the neural network
transform_train = transforms.Compose([
    transforms.Resize((image_size, image_size)),                                    # Resize images to 224x224 pixels
    transforms.RandomCrop(size=image_size),                                         # Randomly crop and resize images to 224x224 pixels
    transforms.RandomHorizontalFlip(),                                              # Randomly flip the images horizontally
    transforms.RandomRotation(30),                                                  # Randomly rotate the images by 30 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Randomly adjust brightness, contrast, saturation, and hue
    transforms.ToTensor(),                                                          # Convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])                 # Normalize the images
])

# Define the transformations for test data before entering the neural network
transform_test = transforms.Compose([
    transforms.Resize((image_size, image_size)),                     # Resize images to 224x224 pixels
    transforms.ToTensor(),                                           # Convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the images
])

# Load the training and test datasets
train_dataset = ImageFolder(train_dir, transform=transform_train)

# Create DataLoaders for training, validation, and test datasets
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Split the dataset into training, validation, and test sets
dataset_size = len(train_dataset)
train_size = int(train_ratio * dataset_size)
val_size = int(val_ratio * dataset_size)
test_size = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    train_dataset, [train_size, val_size, test_size])

# Create DataLoaders for managing the data batches
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Resnet model
model = resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)
model = model.to(device)

# Define the loss function, optimizer, and learning rate scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

# Training function
def train(model, criterion, optimizer, train_loader, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()

    accuracy = 100.0 * correct / total
    return running_loss / len(train_loader), accuracy


# Test function for validation and test sets
def test(model, criterion, loader, device):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            running_loss += loss.item()

    accuracy = 100.0 * correct / total
    return running_loss / len(loader), accuracy

# Initialize the best loss variable with infinity
best_loss = float('inf')

# Training loop
for epoch in range(num_epochs):
    train_loss, train_accuracy = train(model, criterion, optimizer, train_loader, device)
    test_loss, test_accuracy = test(model, criterion, test_loader, device)
    val_loss, val_accuracy = test(model, criterion, val_loader, device)

    scheduler.step(test_loss) 

    scheduler.step(test_loss) 

    print(f'Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')
    
    # save model 
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(model.state_dict(), 'best_model_1.pth')

data = {
    "Model type": "resnet18",
    "Dataset use": os.path.basename(train_dir),
    "Image Resize": str(image_size)+"*"+str(image_size),
    "Epochs": num_epochs,
    "Learning rate": learning_rate,
    "Batch size": batch_size,
    "Train Accuracy": f"{train_accuracy:.2f}",
    "Validation accuracy": f"{val_accuracy:.2f}",
    "Test Accuracy": f"{test_accuracy:.2f}",
}

# Check if the CSV file already exists
if os.path.isfile("model_data.csv"):
    existing_data = pd.read_csv("model_data.csv")
    new_data = pd.concat([existing_data, pd.DataFrame(data, index=[0])], ignore_index=True)
else:
    new_data = pd.DataFrame(data, index=[0])

# Save the updated DataFrame to CSV
new_data.to_csv("model_data.csv", index=False)


cpu


C:\Users\HansW\mitwworkspace\MakeAIWork2\env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HansW\mitwworkspace\MakeAIWork2\env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/32: Train Loss: 1.4859, Test Loss: 5.6444, Test Accuracy: 60.00%
Epoch 2/32: Train Loss: 1.2595, Test Loss: 1.0689, Test Accuracy: 57.14%
Epoch 3/32: Train Loss: 1.2258, Test Loss: 1.4308, Test Accuracy: 68.57%
Epoch 4/32: Train Loss: 0.9565, Test Loss: 1.0039, Test Accuracy: 62.86%
Epoch 5/32: Train Loss: 0.8941, Test Loss: 0.6919, Test Accuracy: 71.43%
Epoch 6/32: Train Loss: 0.8782, Test Loss: 1.6320, Test Accuracy: 51.43%
Epoch 7/32: Train Loss: 0.9902, Test Loss: 0.6374, Test Accuracy: 80.00%
Epoch 8/32: Train Loss: 0.7125, Test Loss: 0.9172, Test Accuracy: 65.71%
Epoch 9/32: Train Loss: 0.6912, Test Loss: 1.4379, Test Accuracy: 57.14%
Epoch 10/32: Train Loss: 0.6407, Test Loss: 0.7660, Test Accuracy: 71.43%
Epoch 11/32: Train Loss: 0.5524, Test Loss: 0.5573, Test Accuracy: 80.00%
Epoch 12/32: Train Loss: 0.4447, Test Loss: 0.5588, Test Accuracy: 85.71%
Epoch 13/32: Train Loss: 0.4644, Test Loss: 0.5361, Test Accuracy: 80.00%
Epoch 14/32: Train Loss: 0.3970, Test Loss: 0.4

In [2]:
import os
import torch
import random
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchsummary import summary
from torchvision.models import resnet18
from torch.optim.lr_scheduler import ReduceLROnPlateau
from datetime import datetime
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

resnet_type = "resnet18"
image_size = 224
batch_size = 8
num_epochs = 20
learning_rate = 0.001
num_classes = 4
start_time = time.time()

train_dir = "./../Project 3/apple_disease_classification/Train"
test_dir = "./../Project 3/apple_disease_classification/Test"

# Define the transformations for train data before entering the neural network
transform_train = transforms.Compose([
    transforms.Resize((image_size, image_size)),                                    # Resize images to 224x224 pixels
    transforms.RandomCrop(size=image_size),                                         # Randomly crop and resize images to 224x224 pixels
    transforms.RandomHorizontalFlip(),                                              # Randomly flip the images horizontally
    transforms.RandomRotation(30),                                                  # Randomly rotate the images by 30 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Randomly adjust brightness, contrast, saturation, and hue
    transforms.ToTensor(),                                                          # Convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])                 # Normalize the images
])

# Define the transformations for test data before entering the neural network
transform_test = transforms.Compose([
    transforms.Resize((image_size, image_size)),                     # Resize images to 224x224 pixels
    transforms.ToTensor(),                                           # Convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the images
])

# Load the training and test datasets
train_dataset = ImageFolder(train_dir, transform=transform_train)

# Create DataLoaders for training, validation, and test datasets
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Split the dataset into training, validation, and test sets
dataset_size = len(train_dataset)
train_size = int(train_ratio * dataset_size)
val_size = int(val_ratio * dataset_size)
test_size = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    train_dataset, [train_size, val_size, test_size])

# Create DataLoaders for managing the data batches
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Resnet model
model = resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

def train(model, criterion, optimizer, train_loader, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()

    accuracy = 100.0 * correct / total
    return running_loss / len(train_loader), accuracy


def test(model, criterion, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            running_loss += loss.item()

    accuracy = 100.0 * correct / total
    return running_loss / len(test_loader), accuracy

def validate(model, criterion, val_loader, device):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            running_loss += loss.item()

    accuracy = 100.0 * correct / total
    return running_loss / len(val_loader), accuracy

best_loss = float('inf')

for epoch in range(num_epochs):
    train_loss, train_accuracy = train(model, criterion, optimizer, train_loader, device)
    test_loss, test_accuracy = test(model, criterion, test_loader, device)
    val_loss, val_accuracy = validate(model, criterion, val_loader, device)

    scheduler.step(test_loss) 

    print(f'Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(model.state_dict(), 'best_model_1.pth')

        
# Set a time and date
now = datetime.now()
timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
print(timestamp)

duration = time.time() - start_time
duration_str = time.strftime("%H:%M:%S", time.gmtime(duration))
print(f"Total duration: {duration_str}")
        
data = {
    "Timestamp": timestamp,
    "Duration": duration_str,
    "Model type": resnet_type,
    "Dataset use": os.path.basename(train_dir),
    "Image Resize": str(image_size)+"*"+str(image_size),
    "Epochs": num_epochs,
    "Learning rate": learning_rate,
    "Batch size": batch_size,
    "Train Accuracy": f"{train_accuracy:.2f}",
    "Validation accuracy": f"{val_accuracy:.2f}",
    "Test Accuracy": f"{test_accuracy:.2f}",
}

# Check if the CSV file already exists
if os.path.isfile("model_data.csv"):
    existing_data = pd.read_csv("model_data.csv")
    new_data = pd.concat([existing_data, pd.DataFrame(data, index=[0])], ignore_index=True)
else:
    new_data = pd.DataFrame(data, index=[0])

# Save the updated DataFrame to CSV
new_data.to_csv("model_data.csv", index=False)


cpu


FileNotFoundError: [WinError 3] Het systeem kan het opgegeven pad niet vinden: './../Project 3/apple_disease_classification/Train'